# Reinforcement Learning for Navigation in Grid-Based Environments: Dynamic Programming (Value Iteration & Policy Iteration)

## Introduction

This project is literally a clone of a previous one, where we used Q-Learning to teach an agent how to move in a 5 x 5 grid world to reach a goal while avoiding obstacles.
However, this time we are using a model-based reinforcement learning approach, specifically Dynamic Programming (DP), instead of a model-free one like Q-Learning.

**The main difference between these two approaches is:**

| Concept | Q-Learning | Dynamic Programming |
| ------- | ---------- | ------------------- |
| Type    | Model-Free RL | Model-Based RL          |
| Requires knowledge of environment?       | No   | yes  |
| Learn by interation?       | Yes (explores & learns) | No (plans everything) |
| Computes       | Q-values         | Value function and optimal policy |
| Behavior       | Learns gradually from experience         | Computes optimal plan directly |

In short, Q-Learning makes the agent learn from experience, so it doesn't know what will happen until it tries. In the other hand, Dynamic Programming already knows how the environement works (the transition and rewards), so it can calculate the optimal policy directly before any movement happens.

In this project, we will implement **Value Iteration** and **Policy Iteration**, two key algorithms used in DP.

## Understanding Dynamic Programming in Reinforcement Learning

Dynamic Programming (DP) is a family of algorithms that solve complex problems by breaking them into smaller overlapping subproblems and solving each subproblem recursively.

In Reinforcement Learning, DP methods are used to compute optimal policies and value functions when we have a complete model of the environment; that is, we know the transition probabilities P(s'/s,a) and rewards R(s,a).

it's aim is to find:
* The optimal value function V*(s) or Q*(s,a)
* The corresponding optimal policy π*

#### The Principle of Optimality

Dynamic Programming relies on Bellman’s Principle of Optimality:

    An optimal policy has the property that, whatever the initial state and initial decision, the remaining decisions must constitute an optimal policy with respect to the state resulting from the first decision.

In simple terms:

    “If I make the best choice now, and then continue making the best choices in the future, I’ll be optimal overall.”

This recursive nature leads directly to the Bellman optimality equations.

#### Main Ideas of Dynamic Programming in RL

DP in RL typically involves fours main ideas or steps. Let's go through each:

* **Policy Evaluation**
The idea here is considering a policy π, and then compute its value function Vπ*(s).
We can solve this iteratively by repeatedly updating V(s) until convergence, what is called Iterative Policy Evaluation.
In other words, we estimate how good the policy is by computing expected returns for each state under that policy.

* **Policy Improvement**
Here, given Vπ*(s), we try to make a better policy π' by choosing in each state the action that seems best according to the current value function. This step greedily improves the policy using the most recent value estimates.
Otherwise, we use the current knowledge of how good each state is to choose better actions.

* **Policy Iteration**
In this step we combine Policy Evaluation and Policy Improvement until convergence:
We start with a random policy, we Evaluate Vπ, then we improve it to get π', and we repeat until the policy stops changing.

* **Value Iteration**
This step is crucial to speed up policy iteration by combining evaluation and improvement into a single step. So instead of alternating evaluation and improvement, it just updates values using the Bellman optimality equation until convergence. 

#### Limitations of DP in RL

Dynamic Programming is foundational but not always practical because it assumes:
1. You know the environment model P(s'/s,a) and R(s,a).
2. The state and action spaces are small enough to compute and store tables of values.

That's why in modern RL, sampling-based methods (like Monte Carlo, TD, and Q-Learning) are used to approximate DP's behavior without knowing the full model.

## Value Iteration Implementation

In [8]:
import numpy as np

# Define the Environment Setup
grid_size = (5, 5)
goal_state = (4, 4)
obstacles= [(4, 2), (2, 3)]
actions = ['U', 'D', 'R', 'L']
gamma = 0.9
epsilon = 1e-4

# Define the Reward Function
def get_reward(next_state):
    if next_state == goal_state:
        return 1.0
    elif next_state in obstacles or not (0 <= next_state[0] < grid_size[0] and 0 <= next_state[1] < grid_size[1]):
        return -1.0
    else:
        return -0.05
    
# Define the Transition Model
def get_next_state(state, action):
    i, j = state
    if action == 'U':
        next_state = (i -1, j)
    elif action == 'D':
        next_state = (i + 1, j)
    elif action == 'R':
        next_state = (i, j + 1)
    elif action == 'L':
        next_state = (i, j - 1)
    else:
        next_state = state

    if next_state in obstacles or not (0 <= next_state[0] < grid_size[0] and 0 <= next_state[1] < grid_size[1]):
        next_state = state
    
    return next_state

# Initialize Value Function
V = np.zeros(grid_size)

def value_iteration():
    iteration = 0
    while True:
        delta = 0
        new_V = np.copy(V)
        for i in range(grid_size[0]):
            for j in range(grid_size[1]):
                state = (i, j)
                if state == goal_state or state in obstacles:
                    continue

                value_list = []
                for action in actions:
                    next_state = get_next_state(state, action)
                    r = get_reward(next_state)
                    value = r + gamma * V[next_state]
                    value_list.append(value)
                new_V[state] = max(value_list)
                delta = max(delta, abs(new_V[state] - V[state]))
        
        V[:] = new_V
        iteration += 1
        if delta < epsilon:
            break
    return iteration

iterations = value_iteration()

# Derive the Optimal Policy
policy = np.full(grid_size, ' ')
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        state = (i, j)
        if state == goal_state:
            policy[state] = 'G'
        elif state in obstacles:
            policy[state] = 'X'
        else:
            best_action = None
            best_value = -np.inf
            for action in actions:
                next_state = get_next_state(state, action)
                r = get_reward(next_state)
                value = r + gamma * V[next_state]
                if value > best_value:
                    best_value = value
                    best_action = action
            policy[state] = best_action

print("Policy Iteration Converged in", iterations, "iterations")
print("\nValue Function after Convergence:")
print(V)
print("\nOptimal Policy:")
print(policy)

Policy Iteration Converged in 9 iterations

Value Function after Convergence:
[[0.21744535 0.2971615  0.385735   0.48415    0.5935    ]
 [0.2971615  0.385735   0.48415    0.5935     0.715     ]
 [0.385735   0.48415    0.5935     0.         0.85      ]
 [0.48415    0.5935     0.715      0.85       1.        ]
 [0.385735   0.48415    0.         1.         0.        ]]

Optimal Policy:
[['D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'R' 'D']
 ['D' 'D' 'D' 'X' 'D']
 ['R' 'R' 'R' 'D' 'D']
 ['U' 'U' 'X' 'R' 'G']]


## Policy Iteration Implementation

In [9]:
import numpy as np

# Define the Environment Setup
grid_size = (5, 5)
goal_state = (4, 4)
obstacles= [(4, 2), (2, 3)]
actions = ['U', 'D', 'R', 'L']
gamma = 0.9
epsilon = 1e-4

# Define the Reward Function
def get_reward(next_state):
    if next_state == goal_state:
        return 1.0
    elif next_state in obstacles or not (0 <= next_state[0] < grid_size[0] and 0 <= next_state[1] < grid_size[1]):
        return -1.0
    else:
        return -0.05
    
# Define the Transition Model
def get_next_state(state, action):
    i, j = state
    if action == 'U':
        next_state = (i -1, j)
    elif action == 'D':
        next_state = (i + 1, j)
    elif action == 'R':
        next_state = (i, j + 1)
    elif action == 'L':
        next_state = (i, j - 1)
    else:
        next_state = state

    if next_state in obstacles or not (0 <= next_state[0] < grid_size[0] and 0 <= next_state[1] < grid_size[1]):
        next_state = state
    
    return next_state

def policy_evaluation(policy, V):
    while True:
        delta = 0
        for i in range(grid_size[0]):
            for j in range(grid_size[1]):
                state = (i, j)
                if state == goal_state or state in obstacles:
                    continue
                action = policy[state]
                next_state = get_next_state(state, action)
                r = get_reward(next_state)
                new_value = r + gamma * V[next_state]
                delta = max(delta, abs(V[state] - new_value))
                V[state] = new_value
        if delta < epsilon:
            break
    return V

def policy_improvement(V, policy):
    policy_stable = True
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            state = (i, j)
            if state == goal_state or state in obstacles:
                continue
            old_action = policy[state]
            best_action = None
            best_value = -np.inf
            for action in actions:
                next_state = get_next_state(state, action)
                r = get_reward(next_state)
                value = r + gamma * V[next_state]
                if value > best_value:
                    best_value = value
                    best_action = action
            policy[state] = best_action
            if old_action != best_action:
                policy_stable = False
    return policy, policy_stable

# Initialize random policy
policy = np.random.choice(actions, grid_size)
V = np.zeros(grid_size)

# Mark goal and obstacle states in policy grid
policy[goal_state] = 'G'
for obs in obstacles:
    policy[obs] = 'X'

# Policy Iteration loop
iteration = 0
while True:
    iteration += 1
    V = policy_evaluation(policy, V)
    policy, stable = policy_improvement(V, policy)
    if stable:
        break

print("Policy Iteration Converged in", iteration, "iterations")
print("\nFinal Value Function:\n", V)
print("\nOptimal Policy:\n", policy)

Policy Iteration Converged in 8 iterations

Final Value Function:
 [[0.21744535 0.2971615  0.385735   0.48415    0.5935    ]
 [0.2971615  0.385735   0.48415    0.5935     0.715     ]
 [0.385735   0.48415    0.5935     0.         0.85      ]
 [0.48415    0.5935     0.715      0.85       1.        ]
 [0.385735   0.48415    0.         1.         0.        ]]

Optimal Policy:
 [['D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'R' 'D']
 ['D' 'D' 'D' 'X' 'D']
 ['R' 'R' 'R' 'D' 'D']
 ['U' 'U' 'X' 'R' 'G']]


# Conclusion

In this project, we solved a 5×5 Grid World environment using Dynamic Programming, which is a model-based reinforcement learning approach.

Key Takeaways:

* Dynamic Programming needs complete knowledge of the environment’s model.
* Value Iteration and Policy Iteration are two main DP algorithms for finding optimal policies.
* Both algorithms rely on the Bellman equations to compute value functions.
* Unlike Q-Learning, which learns through exploration, DP methods plan ahead using a known model.
* The optimal policy derived from DP can be visualized to show the best path from the start to the goal.

In summary, Dynamic Programming acts like a planner, while Q-Learning acts like a learner.
Both aim for the same goal, that is to find the best way for an agent to act, but they use very different paths to get there.